In [1]:
from QHyper.solvers.quantum_annealing.advantage import Advantage

import numpy as np
import networkx as nx
from collections import defaultdict

from QHyper.converter import Converter
from QHyper.constraint import Polynomial

from dwave.system import DWaveSampler, EmbeddingComposite
from dimod import BinaryQuadraticModel
from dimod.sampleset import SampleSet

#### Load the biggest graph
where the time differences shall be the biggest

In [2]:
Graphs = np.load("networks/powerlaw_m=1_p=0.2/graphs.npy", allow_pickle=True)
G = Graphs[9]

Prepare dirs

In [3]:
import os


dir = "networks"
os.makedirs(os.path.dirname(dir+"/"), exist_ok=True)


def make_dir(dir):
    if not os.path.exists(dir):
        os.mkdir(dir)

In [4]:
# output folder
dir =dir+"/output"
os.makedirs(os.path.dirname(dir+"/"), exist_ok=True)

In [5]:
import numpy as np


MIN_NODES = 10
MAX_NODES = 100

num_nodes = np.linspace(MIN_NODES, MAX_NODES, MAX_NODES//MIN_NODES)
num_nodes

array([ 10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100.])

#### Advantage sampler with EmeddingComposite (standard)

In [6]:
solver = "adv"
make_dir(f"{dir}/{solver}")

In [7]:
from Qommunity.samplers.hierarchical.advantage_sampler import AdvantageSampler
from Qommunity.searchers.hierarchical_community_searcher.hierarchical_community_searcher import HierarchicalCommunitySearcher

In [15]:
from tqdm import tqdm


N_RUNS = 10

mods = np.zeros((N_RUNS))
comms = np.empty((N_RUNS), dtype=object)

mods_graph_N = np.zeros((N_RUNS))
comms_graph_N = np.empty((N_RUNS), dtype=object)

advantage = AdvantageSampler(G, num_reads=300)
hierch_searcher = HierarchicalCommunitySearcher(advantage)

current_net_size = int(G.number_of_nodes())

for r in tqdm(range(N_RUNS)):
    comms_res = hierch_searcher.hierarchical_community_search()
    mod_score = nx.community.modularity(G, comms_res)
    print(mod_score)
    mods_graph_N[r] = mod_score
    comms_graph_N[r] = comms_res

    # For bigger graphs computations take time, so better save it up
    # even for each run
    try:
        np.save(f"{dir}/{solver}/{solver}-network_size_{current_net_size}", mods_graph_N)
        np.save(f"{dir}/{solver}/{solver}-network_size_{current_net_size}_comms", comms_graph_N)
    except Exception as e:
        print(f"iter {i} run {r} npy saving failed:\n{e}")

mods = mods_graph_N
comms = comms_graph_N

np.save(f"{dir}/{solver}/{solver}-mods_final", mods)
np.save(f"{dir}/{solver}/{solver}-comms_final", comms)

 10%|█         | 1/10 [01:55<17:15, 115.07s/it]

0.7308437914498521


 20%|██        | 2/10 [04:38<19:08, 143.58s/it]

0.7075808590960107


 30%|███       | 3/10 [07:12<17:18, 148.39s/it]

...

0.7180899908172635


 90%|█████████ | 9/10 [26:36<03:21, 201.49s/it]

0.7421691664115906


100%|██████████| 10/10 [30:34<00:00, 183.47s/it]

0.7309458218549129


Time of 10 iterations of graph of size 100 -> 30:34

#### Advantage sampler with FixedEmeddingComposite with clique embedding

In [6]:
solver = "adv_clique_embedding"
make_dir(f"{dir}/{solver}")

Clear all previously stored caches

In [8]:
from minorminer.busclique import busgraph_cache

busgraph_cache.clear_all_caches()

In [7]:
from Qommunity.samplers.hierarchical.advantage_sampler_clique_embedding import AdvantageSamplerCliqueEmbedding
from Qommunity.searchers.hierarchical_community_searcher.hierarchical_community_searcher import HierarchicalCommunitySearcher

In [9]:
from tqdm import tqdm


N_RUNS = 10

mods = np.zeros((N_RUNS))
comms = np.empty((N_RUNS), dtype=object)

mods_graph_N = np.zeros((N_RUNS))
comms_graph_N = np.empty((N_RUNS), dtype=object)

advantage = AdvantageSamplerCliqueEmbedding(G, num_reads=300)
hierch_searcher = HierarchicalCommunitySearcher(advantage)

current_net_size = int(G.number_of_nodes())

for r in tqdm(range(N_RUNS)):
    comms_res = hierch_searcher.hierarchical_community_search()
    mod_score = nx.community.modularity(G, comms_res)
    print(mod_score)
    mods_graph_N[r] = mod_score
    comms_graph_N[r] = comms_res

    # For bigger graphs computations take time, so better save it up
    # even for each run
    try:
        np.save(f"{dir}/{solver}/{solver}-network_size_{current_net_size}", mods_graph_N)
        np.save(f"{dir}/{solver}/{solver}-network_size_{current_net_size}_comms", comms_graph_N)
    except Exception as e:
        print(f"iter {i} run {r} npy saving failed:\n{e}")

mods = mods_graph_N
comms = comms_graph_N

np.save(f"{dir}/{solver}/{solver}-mods_final", mods)
np.save(f"{dir}/{solver}/{solver}-comms_final", comms)

comm: 100 elapsed: 149.18013429641724


  0%|          | 0/10 [00:00<?, ?it/s]

comm: 100 elapsed: 0.14202308654785156
comm: 54 elapsed: 0.11319494247436523
comm: 30 elapsed: 0.1294546127319336
comm: 24 elapsed: 0.12482571601867676
comm: 21 elapsed: 0.10785365104675293
comm: 3 elapsed: 0.20559358596801758
comm: 6 elapsed: 0.42476463317871094
comm: 3 elapsed: 0.20240044593811035
comm: 3 elapsed: 0.22817754745483398
comm: 24 elapsed: 0.19646978378295898
comm: 12 elapsed: 0.15094208717346191
comm: 7 elapsed: 0.2526118755340576
comm: 5 elapsed: 0.5934267044067383
comm: 12 elapsed: 0.16177129745483398
comm: 6 elapsed: 0.1739974021911621
comm: 6 elapsed: 0.14893150329589844
comm: 5 elapsed: 0.18194103240966797
comm: 46 elapsed: 0.20196080207824707
comm: 26 elapsed: 0.15453600883483887
comm: 5 elapsed: 0.18084359169006348
comm: 21 elapsed: 0.1510148048400879
comm: 4 elapsed: 0.15668749809265137
comm: 17 elapsed: 0.16309475898742676
comm: 4 elapsed: 0.2254323959350586
comm: 13 elapsed: 0.16528820991516113
comm: 20 elapsed: 0.27760887145996094
comm: 9 elapsed: 0.1644835472

 10%|█         | 1/10 [00:48<07:14, 48.25s/it]

0.7284970921334558
comm: 100 elapsed: 0.39829516410827637
comm: 53 elapsed: 0.44321775436401367
comm: 24 elapsed: 0.17559504508972168
comm: 12 elapsed: 0.17530298233032227
comm: 6 elapsed: 0.18807101249694824
comm: 5 elapsed: 0.16643905639648438
comm: 6 elapsed: 0.1692674160003662
comm: 12 elapsed: 0.17169570922851562
comm: 7 elapsed: 0.16972613334655762
comm: 5 elapsed: 0.15020513534545898
comm: 29 elapsed: 0.14836716651916504
comm: 23 elapsed: 0.2859025001525879
comm: 20 elapsed: 0.2289431095123291
comm: 3 elapsed: 0.16842174530029297
comm: 6 elapsed: 0.1717674732208252
comm: 3 elapsed: 0.15079450607299805
comm: 3 elapsed: 0.1532735824584961
comm: 47 elapsed: 0.14154911041259766
comm: 26 elapsed: 0.22613811492919922
comm: 5 elapsed: 0.14932775497436523
comm: 21 elapsed: 0.1362617015838623
comm: 4 elapsed: 0.2280285358428955
comm: 17 elapsed: 0.1391124725341797
comm: 4 elapsed: 0.22115731239318848
comm: 13 elapsed: 0.16356301307678223
comm: 21 elapsed: 0.1776881217956543
comm: 10 elap

...

0.7347719620446893
comm: 100 elapsed: 0.40256285667419434
comm: 50 elapsed: 0.388394832611084
comm: 29 elapsed: 0.309112548828125
comm: 23 elapsed: 0.3366086483001709
comm: 20 elapsed: 0.29030919075012207
comm: 3 elapsed: 0.35326313972473145
comm: 6 elapsed: 0.3385124206542969
comm: 3 elapsed: 0.33696508407592773
comm: 3 elapsed: 0.3428668975830078
comm: 21 elapsed: 0.3765990734100342
comm: 11 elapsed: 0.27166056632995605
comm: 5 elapsed: 0.2685434818267822
comm: 4 elapsed: 0.40346217155456543
comm: 6 elapsed: 0.3373703956604004
comm: 10 elapsed: 0.2642695903778076
comm: 5 elapsed: 0.39545440673828125
comm: 5 elapsed: 0.3721811771392822
comm: 50 elapsed: 0.3556993007659912
comm: 24 elapsed: 0.3569905757904053
comm: 6 elapsed: 0.37421464920043945
comm: 5 elapsed: 0.34317660331726074
comm: 18 elapsed: 0.3410074710845947
comm: 14 elapsed: 0.38126683235168457
comm: 4 elapsed: 0.3416128158569336
comm: 26 elapsed: 0.3329169750213623
comm: 7 elapsed: 0.3338899612426758
comm: 19 elapsed: 0.339

 90%|█████████ | 9/10 [10:36<01:10, 70.40s/it]

0.7318130802979288
comm: 100 elapsed: 0.4065089225769043
comm: 54 elapsed: 0.33191514015197754
comm: 27 elapsed: 0.3356504440307617
comm: 22 elapsed: 0.34037184715270996
comm: 4 elapsed: 0.41834545135498047
comm: 18 elapsed: 0.468930721282959
comm: 14 elapsed: 0.5994784832000732
comm: 4 elapsed: 0.5323982238769531
comm: 5 elapsed: 0.466167688369751
comm: 27 elapsed: 0.5242080688476562
comm: 8 elapsed: 0.4615004062652588
comm: 6 elapsed: 0.3947105407714844
comm: 2 elapsed: 0.38393521308898926
comm: 19 elapsed: 0.31831789016723633
comm: 8 elapsed: 0.3442234992980957
comm: 11 elapsed: 0.36658644676208496
comm: 46 elapsed: 0.34314608573913574
comm: 16 elapsed: 0.38835644721984863
comm: 6 elapsed: 0.5361676216125488
comm: 10 elapsed: 0.39650750160217285
comm: 5 elapsed: 0.3709108829498291
comm: 5 elapsed: 0.38314294815063477
comm: 30 elapsed: 0.40004682540893555
comm: 24 elapsed: 0.3843364715576172
comm: 21 elapsed: 0.4232001304626465
comm: 3 elapsed: 0.5272345542907715
comm: 6 elapsed: 0.4

100%|██████████| 10/10 [11:47<00:00, 70.77s/it]

0.7381389654116929


Time of 10 iterations of graph of size 100 -> 11:47 \
quite big time saving thanks to caching.

#### Results

Quick leiden runs to be able to compare modularities

In [11]:
from samplers.regular.leiden_sampler.leiden_sampler import LeidenSampler
from searchers.community_searcher.community_searcher import CommunitySearcher


N_RUNS = 10

leiden = LeidenSampler(G)
searcher = CommunitySearcher(leiden)

mods_leiden = []
for i in range(N_RUNS):
    res = searcher.community_search()
    mod = nx.community.modularity(G, res)
    mods_leiden.append(mod)

mods_leiden

[0.7447199265381086,
 0.7447199265381086,
 0.7447199265381086,
 0.7447199265381086,
 0.7447199265381086,
 0.7447199265381086,
 0.7447199265381086,
 0.7447199265381086,
 0.7447199265381086,
 0.7447199265381086]

Modularities with clique embedding

In [24]:
mods_cl = mods
mods_adv = np.load(f"{dir}/adv/adv-mods_final.npy")

In [15]:
print(np.mean(mods_cl))
print(np.max(mods_cl))

0.7317569635751455
0.7403326191204981


Modularities with standard embedding

In [16]:
print(np.mean(mods_adv))
print(np.max(mods_adv))

0.7307621671258036
0.7376798285889196
